In [1]:
import os
from langchain_openai import ChatOpenAI
from getpass import getpass

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

chat = ChatOpenAI(
    openai_api_key=os.environ["OPENAI_API_KEY"],
    model='gpt-3.5-turbo-0125'
)

In [2]:
from langchain.schema import (
    SystemMessage,
    HumanMessage,
    AIMessage
)

messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="Hi AI, how are you today?"),
    AIMessage(content="I'm great thank you. How can I help you?"),
    HumanMessage(content="I'd like to understand string theory.")
]

In [3]:
res = chat(messages)


/tmp/ipykernel_14280/3952239659.py:1: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  res = chat(messages)


In [4]:
print(res.content)

String theory is a theoretical framework in physics that attempts to reconcile quantum mechanics and general relativity. It proposes that the fundamental building blocks of the universe are not particles like electrons or quarks, but rather tiny vibrating strings. These strings can vibrate at different frequencies, giving rise to different particles and forces that we observe in nature.

String theory also suggests the existence of extra dimensions beyond the familiar three dimensions of space and one dimension of time. These extra dimensions are compactified or curled up at extremely small scales, making them effectively invisible at everyday scales.

String theory has the potential to unify all fundamental forces of nature, including gravity, electromagnetism, the weak nuclear force, and the strong nuclear force, into a single theoretical framework. However, it is still a work in progress and has not yet been experimentally confirmed. Scientists continue to explore the implications a

In [5]:
# add latest AI response to messages
messages.append(res)

# now create a new user prompt
prompt = HumanMessage(
    content="Why do physicists believe it can produce a 'unified theory'?"
)
# add to messages
messages.append(prompt)

# send to chat-gpt
res = chat(messages)

print(res.content)

Physicists believe that string theory has the potential to produce a unified theory because it offers a way to describe all fundamental forces and particles within a single theoretical framework. Traditional theories of physics, such as quantum mechanics and general relativity, have been successful in describing specific aspects of the universe at different scales. However, these theories are not compatible with each other in all situations, leading to inconsistencies and limitations when trying to describe phenomena at very small scales or under extreme conditions.

String theory, on the other hand, aims to reconcile these seemingly contradictory theories by proposing that the fundamental building blocks of the universe are tiny vibrating strings. By incorporating the principles of quantum mechanics and general relativity into a single framework based on string theory, physicists hope to resolve the inconsistencies between the two and provide a more comprehensive understanding of the 

### If you ask the chatGPT model about the latest model, for exmaple Deepseek R1, the model would fail to update anything after it has been trained.

In [6]:
# add latest AI response to messages
messages.append(res)

# now create a new user prompt
prompt = HumanMessage(
    content="What is so special about Deepseek R1?"
)
# add to messages
messages.append(prompt)

# send to OpenAI
res = chat(messages)

In [7]:
print(res.content)

I'm not familiar with Deepseek R1. Could you provide more context or information about what it is?


We can feed additional information to the LLM with some instructions to tell the model how we would like to use this information alogside our original questioin.

In [8]:
source_knowledge = (
    "We introduce our first-generation reasoning models, DeepSeek-R1-Zero and "
    "DeepSeek-R1. DeepSeek-R1-Zero, a model trained via large-scale "
    "reinforcement learning (RL) without supervised fine-tuning (SFT) as a "
    "preliminary step, demonstrates remarkable reasoning capabilities. Through "
    "RL, DeepSeek-R1-Zero naturally emerges with numerous powerful and "
    "intriguing reasoning behaviors. However, it encounters challenges such as "
    "poor readability, and language mixing. To address these issues and "
    "further enhance reasoning performance, we introduce DeepSeek-R1, which "
    "incorporates multi-stage training and cold-start data before RL. "
    "DeepSeek-R1 achieves performance comparable to OpenAI-o1-1217 on "
    "reasoning tasks. To support the research community, we open-source "
    "DeepSeek-R1-Zero, DeepSeek-R1, and six dense models (1.5B, 7B, 8B, 14B, "
    "32B, 70B) distilled from DeepSeek-R1 based on Qwen and Llama."
)

In [9]:
query = "What is so special about Deepseek R1?"

augmented_prompt = f"""Using the contexts below, answer the query.

Contexts:
{source_knowledge}

Query: {query}"""

In [10]:
# create a new user prompt
prompt = HumanMessage(
    content=augmented_prompt
)
# add to messages
messages.append(prompt)

# send to OpenAI
res = chat(messages)

In [11]:
print(res.content)

DeepSeek R1 is a special deep learning model that stands out for its impressive reasoning capabilities. It is designed to address challenges encountered by its predecessor, DeepSeek-R1-Zero, such as poor readability and language mixing, by incorporating multi-stage training and cold-start data before reinforcement learning (RL). This approach has led to DeepSeek R1 achieving performance comparable to OpenAI-o1-1217 on reasoning tasks. Additionally, the developers have open-sourced DeepSeek-R1-Zero, DeepSeek-R1, and six dense models distilled from DeepSeek-R1 to support the research community.


### Importing the Data

In this task, we will be importing our data. We will be using the Hugging Face Datasets library to load our data. Specifically, we will be using the "jamescalam/deepseek-r1-paper-chunked" dataset. This dataset contains the Deepseek R1 paper pre-processed into RAG-ready chunks.

In [12]:
from datasets import load_dataset

dataset = load_dataset(
    "jamescalam/deepseek-r1-paper-chunked",
    split="train"
)

dataset

/home/philippe/miniconda3/envs/rag/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating train split: 100%|██████████| 76/76 [00:00<00:00, 18999.11 examples/s]


Dataset({
    features: ['doi', 'chunk-id', 'chunk', 'num_tokens', 'pages', 'source'],
    num_rows: 76
})

In [ ]:
my_list: list[str] =  ["1", 2, "3"]
my_list2: list[int] = ["1", "3"]



[[[0, 0], [0, 0], [0, 0]],
 [[0, 0], [0, 0], [0, 0]],
 [[0, 0], [0, 0], [0, 0]],
 [[0, 0], [0, 0], [0, 0]],
 [[0, 0], [0, 0], [0, 0]]]

In [1]:
8 // 4

2